<a href="https://www.kaggle.com/joachimrives/warthunder-performance-analysis?scriptVersionId=86598917" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/warthunder-manual-killdeath-record/records.csv


# WarThunder Performance Analysis

In [2]:
import pandas as pd
import numpy as np
import pprint
import re
import Levenshtein
import datetime as dt

### Current Goal:
#### Nov. 04, 2021-11-04
- Correct misspelled vehicle names based on string similarity and frequencies. Steps:
1. First step is to identify names that rarely appear. It is assumed that these are misspelled names.
2. For each mispelled name, find the most similar name that appears frequently.

- Hamming distance is not appropriate since it only works on strings with the same length.
    
- The Levenshtein Distance method cannot distinguish well between missing characters and coincidentally similar names. It might work if there are no extremly similar and rare correct names. E.g. T-55AM-1 versus T55AM-1 will be parsed correctly provided only one character is misspelled.

- Levenshtein distance cannot distinguish between a swapping error and similar but clearly different names, e.g. 1949 vs. 1974 and 1947 vs 1974 have the same Levenshtein distance.


Sources:

Comparison of edit-distance methods for measuring string similarity:
https://www.baeldung.com/cs/string-similarity-edit-distance

String similarity calculations for merging different versions of each app.
https://medium.com/@appaloosastore/string-similarity-algorithms-compared-3f7b4d12f0ff

Built-in Python libraries for Levenshtein and Cosine Similarity:
https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a

Manual implementation of Hamming Distance and Levenshtein Distance:
https://www.analyticsvidhya.com/blog/2021/02/a-simple-guide-to-metrics-for-calculating-string-similarity/

### Testing Levenshtein Distance

In [3]:
print(Levenshtein.distance("T-55AM-1", "T55AM-1"))

1


In [4]:
print(Levenshtein.distance("T-62M-1", "T55AM-1"))

3


In [5]:
print(Levenshtein.distance("T-55A", "T55AM-1"))

4


In [6]:
print(Levenshtein.distance("T-54 (1947)", "T-54(1947)"))

1


In [7]:
print(Levenshtein.distance("T-54 (1949)", "T-54(1947)"))

2


In [8]:
print(Levenshtein.distance("T-54 (1949)", "T-54 (1974)"))

2


In [9]:
print(Levenshtein.distance("T-54 (1947)", "T-54 (1974)"))

2


In [10]:
print(Levenshtein.distance("T-54 (1949)", "T54(1947)"))

3


In [11]:
df_data = pd.read_csv("../input/warthunder-manual-killdeath-record/records.csv")
df_data.head()

,DateRecorded,VehicleUsed,Event,EnemyVehicle,Distance,OrientationInSight,EnemySpeed,SelfSpeed
0,2021-12-05,8.8 cm Flak 37 Sfzl,k,T-34 (1942),1.24,Diag,0,0
1,2021-12-05,8.8 cm Flak 37 Sfzl,k,T-34 China,1.16,Near-para,0,0
2,2021-12-30,Centurion Mk. 10,k,IS-3,.67,Para,8,0
3,2021-12-30,Centurion Mk. 10,k,IS-4M,.67,Perp,0,0
4,2021-12-30,Centurion Mk. 10,k,PT-76,.68,Diag,0,0


In [12]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2548 entries, 0 to 2547
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   DateRecorded        2548 non-null   object
 1   VehicleUsed         2548 non-null   object
 2   Event               2548 non-null   object
 3   EnemyVehicle        2540 non-null   object
 4   Distance            2524 non-null   object
 5   OrientationInSight  2511 non-null   object
 6   EnemySpeed          2484 non-null   object
 7   SelfSpeed           2435 non-null   object
dtypes: object(8)
memory usage: 159.4+ KB


In [13]:
df_data.describe()

,DateRecorded,VehicleUsed,Event,EnemyVehicle,Distance,OrientationInSight,EnemySpeed,SelfSpeed
count,2548,2548,2548,2540,2524,2511,2484,2435
unique,212,19,33,270,351,61,130,77
top,2021-12-30,Centurion Mk. 10,k,T-54 (1949),.2,Diag,0,0
freq,77,1324,1798,335,71,986,1534,1985


# Data Cleaning

- Columns that documentation says are numberic should also be identified by Pandas as numeric

In [14]:
df_data.dropna(inplace=True)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2430 entries, 0 to 2547
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   DateRecorded        2430 non-null   object
 1   VehicleUsed         2430 non-null   object
 2   Event               2430 non-null   object
 3   EnemyVehicle        2430 non-null   object
 4   Distance            2430 non-null   object
 5   OrientationInSight  2430 non-null   object
 6   EnemySpeed          2430 non-null   object
 7   SelfSpeed           2430 non-null   object
dtypes: object(8)
memory usage: 170.9+ KB


In [15]:
def remove_nondigit(input_string: str):
    return re.sub(pattern="[^0-9.]", repl='', string=input_string)

- Remove non-digit characters from numeric columns before selecting columns to drop. Reduces lost data.

- Reduced dropped rows from 275 to 111, i.e. 1992 - 1717 to 2100 - 1989.

In [16]:
df_data["Distance"] = pd.to_numeric(df_data["Distance"].map(arg=remove_nondigit), errors="coerce")
df_data["EnemySpeed"] = pd.to_numeric(df_data["EnemySpeed"].map(arg=remove_nondigit), errors="coerce")
df_data["SelfSpeed"] = pd.to_numeric(df_data["SelfSpeed"].map(arg=remove_nondigit), errors="coerce")

df_data.dropna(inplace=True)

In [17]:
# bmask_invalid_distance = df_data["Distance"].str.contains("[^0-9.]|\s")
# bmask_invalid_espeed = df_data["EnemySpeed"].str.contains("[^0-9.]|\s")
# bmask_invalid_speed = df_data["SelfSpeed"].str.contains("[^0-9.]|\s")

# lst_invalids = list()
# lst_invalids += list(df_data.loc[bmask_invalid_distance].index)
# lst_invalids += list(df_data.loc[bmask_invalid_espeed].index)
# lst_invalids += list(df_data.loc[bmask_invalid_speed].index)

# df_data.drop(inplace=True, index=lst_invalids)

# print(lst_invalids)

In [18]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2308 entries, 0 to 2547
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DateRecorded        2308 non-null   object 
 1   VehicleUsed         2308 non-null   object 
 2   Event               2308 non-null   object 
 3   EnemyVehicle        2308 non-null   object 
 4   Distance            2308 non-null   float64
 5   OrientationInSight  2308 non-null   object 
 6   EnemySpeed          2308 non-null   float64
 7   SelfSpeed           2308 non-null   float64
dtypes: float64(3), object(5)
memory usage: 162.3+ KB


In [19]:
df_data["Event"] = df_data["Event"].str.lower()

In [20]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2308 entries, 0 to 2547
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DateRecorded        2308 non-null   object 
 1   VehicleUsed         2308 non-null   object 
 2   Event               2308 non-null   object 
 3   EnemyVehicle        2308 non-null   object 
 4   Distance            2308 non-null   float64
 5   OrientationInSight  2308 non-null   object 
 6   EnemySpeed          2308 non-null   float64
 7   SelfSpeed           2308 non-null   float64
dtypes: float64(3), object(5)
memory usage: 162.3+ KB


In [21]:
df_data.describe()

,Distance,EnemySpeed,SelfSpeed
count,2308.000000,2308.000000,2308.000000
mean,0.586350,4.970755,1.111785
std,0.548835,8.968558,4.053402
min,0.010000,0.000000,0.000000
25%,0.180000,0.000000,0.000000
50%,0.500000,0.000000,0.000000
75%,0.880000,7.000000,0.000000
max,11.670000,49.000000,41.000000


### Dealing with Apparently Erroneous Distances

The area in which ground vehicles are allowed to travel within all WarThunder maps never exceeds 3 kilometers. The recommended solution is to remove these columns even if the cuase for typographical errors can be guessed. E.g. double-digit non-decimal numbers or those with special characters corresponding to number keys on a standard keyboard.

In [22]:
bmask_overdistance = df_data["Distance"] > 3
df_data.loc[bmask_overdistance]

,DateRecorded,VehicleUsed,Event,EnemyVehicle,Distance,OrientationInSight,EnemySpeed,SelfSpeed
188,2021-11-24,Centurion Mk. 10,k,SU-100P,7.00,Diag,12.0,3.0
2002,2021-05-11,Tiger E,k,M4A3E2,11.67,Diag,17.0,0.0


In [23]:
df_data.drop(inplace=True, index=df_data.loc[bmask_overdistance].index)

In [24]:
df_data.describe()

,Distance,EnemySpeed,SelfSpeed
count,2306.000000,2306.000000,2306.000000
mean,0.578762,4.962491,1.111448
std,0.479876,8.967750,4.054903
min,0.010000,0.000000,0.000000
25%,0.180000,0.000000,0.000000
50%,0.500000,0.000000,0.000000
75%,0.880000,7.000000,0.000000
max,2.910000,49.000000,41.000000


### Enforcing Data-types Based on Dataset Documentation

#### Distance, SelfSpeed, and EnemySpeed should be floats.

In [25]:
df_data["SelfSpeed"] = df_data["SelfSpeed"].astype(dtype=float)
df_data["EnemySpeed"] = df_data["EnemySpeed"].astype(dtype=float)
df_data["Distance"] = df_data["Distance"].astype(dtype=float)

In [26]:
df_data.describe()

,Distance,EnemySpeed,SelfSpeed
count,2306.000000,2306.000000,2306.000000
mean,0.578762,4.962491,1.111448
std,0.479876,8.967750,4.054903
min,0.010000,0.000000,0.000000
25%,0.180000,0.000000,0.000000
50%,0.500000,0.000000,0.000000
75%,0.880000,7.000000,0.000000
max,2.910000,49.000000,41.000000


In [27]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2306 entries, 0 to 2547
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DateRecorded        2306 non-null   object 
 1   VehicleUsed         2306 non-null   object 
 2   Event               2306 non-null   object 
 3   EnemyVehicle        2306 non-null   object 
 4   Distance            2306 non-null   float64
 5   OrientationInSight  2306 non-null   object 
 6   EnemySpeed          2306 non-null   float64
 7   SelfSpeed           2306 non-null   float64
dtypes: float64(3), object(5)
memory usage: 162.1+ KB


#### DateRecorded is converted to a Date type from a string type.

In [28]:
df_data["DateRecorded"] = pd.to_datetime(df_data["DateRecorded"], format = "%Y-%m-%d", errors = "coerce")
bmask_isnat =  pd.isnull(df_data["DateRecorded"])
df_data.drop(inplace=True, index=df_data.loc[bmask_isnat].index)

In [29]:
df_data.dtypes

DateRecorded          datetime64[ns]
VehicleUsed                   object
Event                         object
EnemyVehicle                  object
Distance                     float64
OrientationInSight            object
EnemySpeed                   float64
SelfSpeed                    float64
dtype: object

# Exploratory Data Analysis

In [30]:
bmask_kb = df_data["Event"] == "kb"
bmask_kill = df_data["Event"] == "k"

**Average distance at which a "killed-by" event occurrs.**

In [31]:
df_data.loc[bmask_kb]["Distance"].describe()

count    572.000000
mean       0.428610
std        0.446039
min        0.010000
25%        0.080000
50%        0.275000
75%        0.660000
max        2.850000
Name: Distance, dtype: float64

**Average distance at which a "kill" is made by the player.**

In [32]:
df_data.loc[bmask_kill]["Distance"].describe()

count    1728.000000
mean        0.628472
std         0.480890
min         0.010000
25%         0.227500
50%         0.560000
75%         0.930000
max         2.910000
Name: Distance, dtype: float64

In [33]:
def make_bmask(columns_and_values : list, df : pd.DataFrame = df_data):
    masks = list()
    
    for col_val_pair in columns_and_values:
        column_name, target_value = col_val_pair[0], col_val_pair[1]
        masks.append(df[column_name].str.contains(pat = target_value, flags = re.I))
    
    return masks


def group_df(
    groupby : list,
    masks : list(tuple()),
    df : pd.DataFrame = df_data,
    keep_columns : list = ["Event", "VehicleUsed", "EnemyVehicle",
                           "Distance", "OrientationInSight", "EnemySpeed", "SelfSpeed"]
    ):
    
    # result = pd.DataFrame(columns = df.columns)
    result = list()
    
    for bmask in masks:
        # return df.loc[bmask, keep_columns].groupby(by = groupby, axis = "rows").mean()
        result.append(
            #df.loc[bmask, keep_columns].groupby(by = groupby, axis = "rows").mean()
            df.loc[bmask, keep_columns].groupby(by = groupby, axis = "rows").agg(func=["count", "mean"])
        )
    
    # return df.loc[masks[0], keep_columns]
    return result

In [34]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2300 entries, 0 to 2547
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DateRecorded        2300 non-null   datetime64[ns]
 1   VehicleUsed         2300 non-null   object        
 2   Event               2300 non-null   object        
 3   EnemyVehicle        2300 non-null   object        
 4   Distance            2300 non-null   float64       
 5   OrientationInSight  2300 non-null   object        
 6   EnemySpeed          2300 non-null   float64       
 7   SelfSpeed           2300 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 161.7+ KB


In [35]:
bmasks = make_bmask(
    columns_and_values = [
        ("VehicleUsed", "firefly"),
        ("VehicleUsed", "sherman"),
        ("VehicleUsed", "centurion mk. 10")
    ]
)

bmask_firefly = df_data["VehicleUsed"].str.contains(pat="firefly", flags=re.I)
bmask_vc = df_data["VehicleUsed"].str.contains(pat="sherman", flags=re.I)
bmask_tiger_e = df_data["VehicleUsed"].str.contains(pat="tiger", flags=re.I)
bmask_cent = df_data["VehicleUsed"].str.contains(pat="centurion mk. 10", flags=re.I)

bmask_enemy85 = df_data["EnemyVehicle"].str.contains(pat="85")
bmask_enemy54 = df_data["EnemyVehicle"].str.contains(pat="54")

standard_mask = [
    bmask_firefly & bmask_enemy85,
    bmask_vc & bmask_enemy85,
    bmask_tiger_e & bmask_enemy85,
    bmask_cent & bmask_enemy54
]

# group_df(
#     groupby = ["Event", "VehicleUsed", "EnemyVehicle"],
#     masks = standard_mask
#     # masks = bmasks
# )

### Mean distance of engagements for Firefly/Sherman Vc vs. 85mm-armed tanks.

In [36]:
bmask_subject_vehicle = df_data["VehicleUsed"].str.contains(pat="fly", flags=re.I)
bmask_target_vehicle = df_data["EnemyVehicle"].str.contains(pat="85", flags=re.I)
df_data_by_vehicle = df_data.loc[bmask_subject_vehicle & bmask_target_vehicle].groupby(by=["Event","EnemyVehicle"], axis="rows").mean()
df_data_by_vehicle

Distance  EnemySpeed  SelfSpeed
Event EnemyVehicle                                    
k     KV-85            0.328000    4.000000   0.000000
      SU-85            0.440000    0.000000   0.000000
      SU-85M           0.740000    4.833333   0.000000
      T-34-85          0.440000    5.222222   0.444444
      T-34-85 (D-5T)   0.425833    8.500000   0.000000
      T-34-85 (S53)    0.720000   13.000000   0.000000
      T-34-85 Gai      0.810000    0.000000   0.000000
      T-34-85E         0.635000    1.500000   0.000000
kb    KV-85            0.253333    1.333333   0.000000
      SU-85            0.800000    0.000000   0.000000
      T-34-85          0.370000    3.000000   2.500000
      T-34-85 (D-5T)   0.172000    5.300000   0.000000
      T-34-85 Gai      0.800000   15.000000   0.000000
      T-34-85 No. 215  0.550000    0.000000   0.000000

In [37]:
bmask_subject_vehicle = df_data["VehicleUsed"].str.contains(pat="sherman", flags=re.I)
bmask_target_vehicle = df_data["EnemyVehicle"].str.contains(pat="85", flags = re.I)
df_data_by_vehicle = df_data.loc[bmask_subject_vehicle & bmask_target_vehicle].groupby(by=["Event","EnemyVehicle"], axis="rows").mean()
df_data_by_vehicle

Distance  EnemySpeed  SelfSpeed
Event EnemyVehicle                                   
k     KV-85           0.480000    0.000000        0.0
      SU-85           0.543333    0.000000        0.0
      SU-85M          1.180000    0.000000        0.0
      T-34-85         0.784286    3.428571        0.0
      T-34-85 (D-5T)  0.338000    6.000000        0.0
kb    KV-85           0.552500    0.000000        0.0
      T-34-85         0.290000    0.000000        2.0
      T-34-85 (D-5T)  0.050000    0.000000        0.0
      T-34-85 ?       0.050000    0.000000        0.0

### Average engagement range of Tiger vs. 85mm-armed tanks.

In [38]:
bmask_subject_vehicle = df_data["VehicleUsed"].str.contains(pat = "tiger", flags = re.I)
bmask_target_vehicle = df_data["EnemyVehicle"].str.lower().str.contains(pat="85")
df_data_by_vehicle = df_data.loc[bmask_subject_vehicle & bmask_target_vehicle].groupby(by=["Event","EnemyVehicle"], axis="rows").mean()
df_data_by_vehicle

Distance  EnemySpeed  SelfSpeed
Event EnemyVehicle                                    
k     KV-85            0.574000    0.800000   0.000000
      SU-85            0.887500    1.375000   0.000000
      SU-85M           0.490000    7.666667   0.000000
      T-34-85          0.612105    4.657895   0.000000
      T-34-85 ( D-5T)  0.300000    0.000000   0.000000
      T-34-85 (D-5T)   0.610000    7.153846   0.730769
      T-34-85 (S-53)   0.145000    6.500000   0.250000
      T-34-85 (S53)    0.320000   10.000000   0.000000
      T-34-85 Gai      0.350000    0.000000   0.000000
kb    SU-85M           0.405000    0.000000   0.000000
      T-34-85          0.273333    0.000000   2.666667
      T-34-85 (D-5T)   0.310000   10.000000   0.500000
      T-34-85 (S-53)   1.280000    0.000000   8.000000
      T-34-85E         0.600000    0.000000   5.000000

### Average engangement distances for Centurion Mk. 10 vs. T-54 series.

In [39]:
bmask_subject_vehicle = df_data["VehicleUsed"].str.contains(pat="Centurion Mk. 10") & df_data["EnemyVehicle"].str.contains(pat="T-54", flags = re.I)
df_data_by_vehicle = df_data.loc[bmask_subject_vehicle].groupby(by=["Event","EnemyVehicle"], axis="rows").mean()
df_data_by_vehicle

Distance  EnemySpeed  SelfSpeed
Event EnemyVehicle                                 
k     T-54 (1947)   0.698459    7.791405   1.330189
      T-54 (1949)   0.713204    7.310204   0.865306
      T-54 (1951)   0.567778    7.777778   1.111111
      T-54 (949)    0.410000    0.000000   0.000000
kb    T-54 (1947)   0.535362    2.159420   2.536232
      T-54 (1949)   0.451883    0.441558   3.798701
      T-54 (1951)   0.105000    0.000000   2.750000

### Longest Kills Made

In [40]:
df_data.loc[(df_data["Distance"] >= 2)].sort_values(by=["Distance"], axis="index", ascending=False)

,DateRecorded,VehicleUsed,Event,EnemyVehicle,Distance,OrientationInSight,EnemySpeed,SelfSpeed
2253,2021-12-28,Sherman Vc,k,M4A4 (SA50),2.91,Near-para,0.0,0.0
57,2021-12-30,Centurion Mk. 10,kb,T-54 (1947),2.85,Near-perp,0.0,0.0
154,2021-12-06,Centurion Mk. 10,k,T-54 (1947),2.75,Diag,0.0,0.0
2252,2021-12-28,Sherman Vc,k,ARL-44,2.53,Diag,31.0,0.0
655,2021-08-02,Centurion Mk. 10,k,IS-6,2.53,Diag,0.0,0.0
1972,2021-06-18,Tiger E,k,Sherman VC,2.40,Diag,0.0,0.0
639,2021-08-02,Centurion Mk. 10,k,SU-100P,2.37,Diag,4.0,0.0
2379,2022-01-19,Centurion Mk. 10,k,T-54 (1949),2.33,Near-Perp,37.0,0.0
2247,2021-12-28,Sherman IC Tipo,k,A.C. IV,2.30,Perp,0.0,0.0
248,2021-11-12,Centurion Mk. 10,k,T-54 (1949),2.29,Near-Para,28.0,0.0


## Kill-Death Ratio Summaries

Recall previous boolean masks: <br>
bmask_kb = df_data["Event"] == "kb" <br>
bmask_kill = df_data["Event"] == "k" <br>

In [41]:
def describe_kd(
    subject_target_pairs : list = [("*", ["*"])],
    df_data = df_data):
    
    # results = dict()
    results = pd.DataFrame()
    
    for tup in subject_target_pairs:
        subject_vehicle = tup[0]
        
        for target_vehicle in tup[1]:
            bmask_subject_vehicle = df_data["VehicleUsed"].str.contains(pat = subject_vehicle)
            bmask_target_vehicle = df_data["EnemyVehicle"].str.contains(pat = target_vehicle, flags = re.I)
            bmask = (bmask_subject_vehicle & bmask_target_vehicle)
            
            # pair_result = df_data.loc[bmask].groupby(by=["Event"]).describe()
            pair_result = df_data.loc[bmask].groupby(by=["Event", "EnemyVehicle"])
            if (len(pair_result) != 0):
                # results[subject_vehicle + ' : ' + target_vehicle] = pair_result.describe()
                results = pd.concat([results, pair_result.describe()])
                
    return results


In [42]:
bmask_62days = df_data["DateRecorded"].dt.date >= (dt.date.today() - dt.timedelta(days=62))
bmask_after_allowskipping = df_data["DateRecorded"].dt.date >= dt.date(2022, 1, 7)
bmask_after_heshfix = df_data["DateRecorded"].dt.date >= dt.date(2021, 12, 21)
bmask_after_wingedlions = df_data["DateRecorded"].dt.date >= dt.date(2021, 12, 13)
bmask_after_newsight_2021_11_12 = df_data["DateRecorded"].dt.date >= dt.date(2021, 11, 12)
bmask_after_respawnrule = df_data["DateRecorded"].dt.date >= dt.date(2021, 10, 4)
bmask_after_redskies = df_data["DateRecorded"].dt.date >= dt.date(2021, 6, 4)

In [43]:
describe_kd(
    subject_target_pairs = [
        ("Centurion Mk. 10", ["1947", "1949"])
    ],
    df_data = df_data.loc[bmask_62days]
)

Distance                                                   \
                      count      mean       std   min     25%    50%     75%   
Event EnemyVehicle                                                             
k     T-54 (1947)      36.0  0.847639  0.557930  0.03  0.5125  0.780  1.0825   
kb    T-54 (1947)      12.0  0.890833  0.732995  0.05  0.4525  0.790  1.1025   
k     T-54 (1949)      60.0  0.711917  0.520451  0.05  0.3700  0.635  0.9575   
kb    T-54 (1949)      16.0  0.379375  0.471812  0.02  0.1200  0.225  0.4250   

                         EnemySpeed            ...               SelfSpeed  \
                     max      count      mean  ...     75%   max     count   
Event EnemyVehicle                             ...                           
k     T-54 (1947)   2.75       36.0  7.425926  ...  13.500  35.0      36.0   
kb    T-54 (1947)   2.85       12.0  2.500000  ...   0.000  30.0      12.0   
k     T-54 (1949)   2.33       60.0  7.300000  ...   7.625  42.0      60.0   
kb    T-54 (1949)   1.82       16.0  0.437500  ...   0.000   7.0      16.0   

                                                                    
                        mean        std  min  25%  50%   75%   max  
Event EnemyVehicle                                                  
k     T-54 (1947)   1.444444   3.706708  0.0  0.0  0.0  0.00  15.0  
kb    T-54 (1947)   1.166667   2.757909  0.0  0.0  0.0  0.00   8.0  
k     T-54 (1949)   1.916667   5.699514  0.0  0.0  0.0  0.00  31.0  
kb    T-54 (1949)   5.375000  10.769556  0.0  0.0  0.0  3.25  32.0  

[4 rows x 24 columns]

In [44]:
describe_kd(
    subject_target_pairs = [
        ("Centurion Mk. 10", ["1947", "1949"])
    ],
    df_data = df_data.loc[bmask_after_allowskipping]
)

Distance                                                 \
                      count      mean       std   min    25%   50%     75%   
Event EnemyVehicle                                                           
k     T-54 (1947)      16.0  0.789687  0.355042  0.09  0.575  0.78  1.0175   
kb    T-54 (1947)       2.0  1.010000  0.084853  0.95  0.980  1.01  1.0400   
k     T-54 (1949)      27.0  0.744074  0.602694  0.05  0.360  0.63  0.9400   
kb    T-54 (1949)       5.0  0.242000  0.164378  0.02  0.180  0.20  0.4000   

                          EnemySpeed            ...               SelfSpeed  \
                      max      count      mean  ...     75%   max     count   
Event EnemyVehicle                              ...                           
k     T-54 (1947)   1.405       16.0  7.093750  ...  10.125  35.0      16.0   
kb    T-54 (1947)   1.070        2.0  0.000000  ...   0.000   0.0       2.0   
k     T-54 (1949)   2.330       27.0  7.537037  ...   7.750  37.0      27.0   
kb    T-54 (1949)   0.410        5.0  0.000000  ...   0.000   0.0       5.0   

                                                                  
                        mean       std  min  25%  50%  75%   max  
Event EnemyVehicle                                                
k     T-54 (1947)   1.125000  2.895399  0.0  0.0  0.0  0.0  11.0  
kb    T-54 (1947)   4.000000  5.656854  0.0  2.0  4.0  6.0   8.0  
k     T-54 (1949)   2.962963  7.261497  0.0  0.0  0.0  0.0  31.0  
kb    T-54 (1949)   3.400000  6.542171  0.0  0.0  0.0  2.0  15.0  

[4 rows x 24 columns]

In [45]:
describe_kd(
    subject_target_pairs = [
        ("Centurion Mk. 10", ["1947", "1949"])
    ],
    df_data = df_data.loc[bmask_after_heshfix]
)

Distance                                                  \
                      count      mean       std   min    25%    50%     75%   
Event EnemyVehicle                                                            
k     T-54 (1947)      26.0  0.783654  0.447132  0.03  0.535  0.780  1.0325   
kb    T-54 (1947)      10.0  0.971000  0.781430  0.05  0.530  0.925  1.1675   
k     T-54 (1949)      48.0  0.720000  0.560615  0.05  0.330  0.615  0.9575   
kb    T-54 (1949)      12.0  0.317500  0.306123  0.02  0.120  0.250  0.4250   

                         EnemySpeed            ...               SelfSpeed  \
                     max      count      mean  ...     75%   max     count   
Event EnemyVehicle                             ...                           
k     T-54 (1947)   1.69       26.0  6.769231  ...  11.875  35.0      26.0   
kb    T-54 (1947)   2.85       10.0  0.000000  ...   0.000   0.0      10.0   
k     T-54 (1949)   2.33       48.0  6.135417  ...   7.000  42.0      48.0   
kb    T-54 (1949)   1.12       12.0  0.583333  ...   0.000   7.0      12.0   

                                                                  
                        mean       std  min  25%  50%  75%   max  
Event EnemyVehicle                                                
k     T-54 (1947)   1.423077  3.336742  0.0  0.0  0.0  0.0  11.0  
kb    T-54 (1947)   1.400000  2.988868  0.0  0.0  0.0  0.0   8.0  
k     T-54 (1949)   2.333333  6.302088  0.0  0.0  0.0  0.0  31.0  
kb    T-54 (1949)   3.916667  9.268307  0.0  0.0  0.0  0.5  30.0  

[4 rows x 24 columns]

In [46]:
describe_kd(
    subject_target_pairs = [
        ("Centurion Mk. 10", ["1947", "1949"])
    ],
    df_data = df_data.loc[bmask_after_wingedlions]
)

Distance                                                \
                      count      mean       std   min    25%   50%    75%   
Event EnemyVehicle                                                          
k     T-54 (1947)      29.0  0.822931  0.441959  0.03  0.550  0.83  1.050   
kb    T-54 (1947)      11.0  0.916364  0.763155  0.05  0.425  0.90  1.135   
k     T-54 (1949)      53.0  0.721226  0.543784  0.05  0.340  0.63  0.980   
kb    T-54 (1949)      13.0  0.312308  0.293688  0.02  0.150  0.25  0.410   

                         EnemySpeed            ...                 SelfSpeed  \
                     max      count      mean  ...       75%   max     count   
Event EnemyVehicle                             ...                             
k     T-54 (1947)   1.69       29.0  6.390805  ...  9.333333  35.0      29.0   
kb    T-54 (1947)   2.85       11.0  2.727273  ...  0.000000  30.0      11.0   
k     T-54 (1949)   2.33       53.0  6.037736  ...  7.000000  42.0      53.0   
kb    T-54 (1949)   1.12       13.0  0.538462  ...  0.000000   7.0      13.0   

                                                                   
                        mean        std  min  25%  50%  75%   max  
Event EnemyVehicle                                                 
k     T-54 (1947)   1.275862   3.183625  0.0  0.0  0.0  0.0  11.0  
kb    T-54 (1947)   1.272727   2.866737  0.0  0.0  0.0  0.0   8.0  
k     T-54 (1949)   2.169811   6.024743  0.0  0.0  0.0  0.0  31.0  
kb    T-54 (1949)   6.076923  11.807212  0.0  0.0  0.0  2.0  32.0  

[4 rows x 24 columns]

In [47]:
describe_kd(
    subject_target_pairs = [
        ("Centurion Mk. 10", ["1947", "1949"])
    ],
    df_data = df_data.loc[bmask_after_respawnrule]
)

Distance                                                  \
                      count      mean       std   min     25%   50%     75%   
Event EnemyVehicle                                                            
k     T-54 (1947)      65.0  0.652385  0.505310  0.03  0.3000  0.58  0.9200   
kb    T-54 (1947)      19.0  0.695789  0.671948  0.02  0.2300  0.61  1.0100   
k     T-54 (1949)     108.0  0.676204  0.544955  0.03  0.2175  0.56  0.9800   
kb    T-54 (1949)      30.0  0.385000  0.484282  0.02  0.0425  0.20  0.4475   

                         EnemySpeed            ...               SelfSpeed  \
                     max      count      mean  ...     75%   max     count   
Event EnemyVehicle                             ...                           
k     T-54 (1947)   2.75       65.0  7.682051  ...  14.000  35.0      65.0   
kb    T-54 (1947)   2.85       19.0  5.105263  ...   3.500  30.0      19.0   
k     T-54 (1949)   2.33      108.0  8.472222  ...  15.625  49.0     108.0   
kb    T-54 (1949)   1.82       30.0  0.233333  ...   0.000   7.0      30.0   

                                                                  
                        mean       std  min  25%  50%  75%   max  
Event EnemyVehicle                                                
k     T-54 (1947)   1.153846  3.217291  0.0  0.0  0.0  0.0  15.0  
kb    T-54 (1947)   2.421053  4.799488  0.0  0.0  0.0  2.0  17.0  
k     T-54 (1949)   1.439815  4.702712  0.0  0.0  0.0  0.0  31.0  
kb    T-54 (1949)   4.000000  8.497464  0.0  0.0  0.0  3.5  32.0  

[4 rows x 24 columns]

In [48]:
describe_kd(
    subject_target_pairs = [
        ("Centurion Mk. 10", ["1947", "1949"])
    ],
    df_data = df_data.loc[bmask_after_newsight_2021_11_12]
)

Distance                                                  \
                      count      mean       std   min     25%   50%     75%   
Event EnemyVehicle                                                            
k     T-54 (1947)      47.0  0.744787  0.550418  0.03  0.3300  0.63  1.0250   
kb    T-54 (1947)      14.0  0.889286  0.679768  0.05  0.5125  0.79  1.0925   
k     T-54 (1949)      73.0  0.671164  0.548140  0.05  0.2200  0.58  0.9000   
kb    T-54 (1949)      20.0  0.385000  0.468767  0.02  0.1225  0.20  0.4250   

                         EnemySpeed            ...                  SelfSpeed  \
                     max      count      mean  ...        75%   max     count   
Event EnemyVehicle                             ...                              
k     T-54 (1947)   2.75       47.0  6.347518  ...   9.166667  35.0      47.0   
kb    T-54 (1947)   2.85       14.0  2.642857  ...   0.000000  30.0      14.0   
k     T-54 (1949)   2.33       73.0  8.082192  ...  14.000000  42.0      73.0   
kb    T-54 (1949)   1.82       20.0  0.350000  ...   0.000000   7.0      20.0   

                                                                    
                        mean        std  min  25%  50%   75%   max  
Event EnemyVehicle                                                  
k     T-54 (1947)   1.595745   3.698763  0.0  0.0  0.0  0.00  15.0  
kb    T-54 (1947)   2.071429   3.689158  0.0  0.0  0.0  3.00  11.0  
k     T-54 (1949)   1.821918   5.470483  0.0  0.0  0.0  0.00  31.0  
kb    T-54 (1949)   5.500000  10.044637  0.0  0.0  0.0  7.25  32.0  

[4 rows x 24 columns]

In [49]:
describe_kd(
    subject_target_pairs = [
        ("Centurion Mk. 10", ["1947", "1949"])
    ],
    df_data = df_data.loc[(~bmask_after_newsight_2021_11_12) & (bmask_after_redskies)]
)

Distance                                                \
                      count      mean       std   min     25%   50%   75%   
Event EnemyVehicle                                                          
k     T-54 (1947)      63.0  0.667778  0.451219  0.03  0.3050  0.58  0.93   
kb    T-54 (1947)      29.0  0.372069  0.387422  0.01  0.0500  0.26  0.58   
k     T-54 (1949)     109.0  0.723991  0.483626  0.02  0.3650  0.64  1.06   
kb    T-54 (1949)      28.0  0.558929  0.539501  0.02  0.0875  0.36  0.97   

                         EnemySpeed            ...             SelfSpeed  \
                     max      count      mean  ...   75%   max     count   
Event EnemyVehicle                             ...                         
k     T-54 (1947)   2.28       63.0  9.642857  ...  17.0  46.0      63.0   
kb    T-54 (1947)   1.47       29.0  3.517241  ...   0.0  22.0      29.0   
k     T-54 (1949)   2.24      109.0  7.899083  ...  15.0  49.0     109.0   
kb    T-54 (1949)   1.84       28.0  0.750000  ...   0.0  21.0      28.0   

                                                                  
                        mean       std  min  25%  50%  75%   max  
Event EnemyVehicle                                                
k     T-54 (1947)   1.253968  5.962161  0.0  0.0  0.0  0.0  33.0  
kb    T-54 (1947)   3.793103  7.508944  0.0  0.0  0.0  4.0  32.0  
k     T-54 (1949)   0.600917  3.520956  0.0  0.0  0.0  0.0  34.0  
kb    T-54 (1949)   2.964286  6.197264  0.0  0.0  0.0  1.0  25.0  

[4 rows x 24 columns]

In [50]:
describe_kd(
    subject_target_pairs = [
        ("Centurion Mk. 10", ["1947", "1949"])
    ],
    df_data = df_data.loc[bmask_after_redskies]
)

Distance                                                   \
                      count      mean       std   min     25%    50%     75%   
Event EnemyVehicle                                                             
k     T-54 (1947)     110.0  0.700682  0.495103  0.03  0.3025  0.600  0.9675   
kb    T-54 (1947)      43.0  0.540465  0.550662  0.01  0.0600  0.460  0.7700   
k     T-54 (1949)     182.0  0.702802  0.509661  0.02  0.3000  0.595  0.9800   
kb    T-54 (1949)      48.0  0.486458  0.513368  0.02  0.0875  0.245  0.7850   

                         EnemySpeed            ...              SelfSpeed  \
                     max      count      mean  ...    75%   max     count   
Event EnemyVehicle                             ...                          
k     T-54 (1947)   2.75      110.0  8.234848  ...  14.00  46.0     110.0   
kb    T-54 (1947)   2.85       43.0  3.232558  ...   0.00  30.0      43.0   
k     T-54 (1949)   2.33      182.0  7.972527  ...  14.75  49.0     182.0   
kb    T-54 (1949)   1.84       48.0  0.583333  ...   0.00  21.0      48.0   

                                                                  
                        mean       std  min  25%  50%  75%   max  
Event EnemyVehicle                                                
k     T-54 (1947)   1.400000  5.101178  0.0  0.0  0.0  0.0  33.0  
kb    T-54 (1947)   3.232558  6.516781  0.0  0.0  0.0  4.0  32.0  
k     T-54 (1949)   1.090659  4.434143  0.0  0.0  0.0  0.0  34.0  
kb    T-54 (1949)   4.020833  8.027849  0.0  0.0  0.0  4.5  32.0  

[4 rows x 24 columns]

In [51]:
describe_kd(
    subject_target_pairs = [
        ("Chieftain Mk. 5", ["T-64A", "T-62M", "T-55AM"])
    ]
)

Distance                                                   \
                      count      mean       std   min     25%    50%     75%   
Event EnemyVehicle                                                             
k     T-64A (1971)     21.0  0.550476  0.382040  0.02  0.2000  0.490  0.8000   
kb    T-64A             2.0  0.505000  0.558614  0.11  0.3075  0.505  0.7025   
      T-64A (1971)     12.0  0.529167  0.663564  0.01  0.0500  0.250  0.8000   
k     T-62M-1          17.0  0.472941  0.405890  0.02  0.1300  0.360  0.7300   
kb    T-62M-1           8.0  0.341250  0.541966  0.02  0.0275  0.070  0.3400   
k     T-55AM-1         13.0  0.710385  0.521993  0.02  0.3100  0.580  1.1700   
kb    T-55AM-1          2.0  0.340000  0.197990  0.20  0.2700  0.340  0.4100   

                         EnemySpeed             ...              SelfSpeed  \
                     max      count       mean  ...    75%   max     count   
Event EnemyVehicle                              ...                          
k     T-64A (1971)  1.23       21.0   6.761905  ...   9.00  31.0      21.0   
kb    T-64A         0.90        2.0  11.500000  ...  17.25  23.0       2.0   
      T-64A (1971)  2.04       12.0   0.000000  ...   0.00   0.0      12.0   
k     T-62M-1       1.23       17.0   9.617647  ...  18.00  37.0      17.0   
kb    T-62M-1       1.40        8.0   1.250000  ...   0.00  10.0       8.0   
k     T-55AM-1      1.49       13.0   5.576923  ...   7.00  23.5      13.0   
kb    T-55AM-1      0.48        2.0   2.500000  ...   3.75   5.0       2.0   

                                                                     
                        mean        std  min   25%  50%   75%   max  
Event EnemyVehicle                                                   
k     T-64A (1971)  2.047619   3.761332  0.0  0.00  0.0  3.00  11.0  
kb    T-64A         2.500000   3.535534  0.0  1.25  2.5  3.75   5.0  
      T-64A (1971)  6.166667  12.481501  0.0  0.00  0.0  5.75  41.0  
k     T-62M-1       1.823529   5.502673  0.0  0.00  0.0  0.00  21.0  
kb    T-62M-1       2.750000   4.131759  0.0  0.00  0.0  5.00  10.0  
k     T-55AM-1      2.230769   3.876292  0.0  0.00  0.0  3.00  10.0  
kb    T-55AM-1      0.000000   0.000000  0.0  0.00  0.0  0.00   0.0  

[7 rows x 24 columns]

In [52]:
describe_kd(
    subject_target_pairs = [
        ("Chieftain Mk. 3", ["T-64A", "T-62M", "T-55AM"])
    ]
)

Distance                                                \
                      count   mean       std   min     25%    50%     75%   
Event EnemyVehicle                                                          
k     T-64A (1971)      2.0  0.515  0.685894  0.03  0.2725  0.515  0.7575   
kb    T-64A (1971)      1.0  0.200       NaN  0.20  0.2000  0.200  0.2000   
k     T-62M-1           1.0  0.580       NaN  0.58  0.5800  0.580  0.5800   
kb    T-55AM-1          2.0  1.160  0.749533  0.63  0.8950  1.160  1.4250   

                         EnemySpeed        ...             SelfSpeed       \
                     max      count  mean  ...   75%   max     count mean   
Event EnemyVehicle                         ...                              
k     T-64A (1971)  1.00        2.0   0.0  ...   0.0   0.0       2.0  0.0   
kb    T-64A (1971)  0.20        1.0   0.0  ...   0.0   0.0       1.0  0.0   
k     T-62M-1       0.58        1.0  17.0  ...  17.0  17.0       1.0  0.0   
kb    T-55AM-1      1.69        2.0   0.0  ...   0.0   0.0       2.0  0.0   

                                                  
                    std  min  25%  50%  75%  max  
Event EnemyVehicle                                
k     T-64A (1971)  0.0  0.0  0.0  0.0  0.0  0.0  
kb    T-64A (1971)  NaN  0.0  0.0  0.0  0.0  0.0  
k     T-62M-1       NaN  0.0  0.0  0.0  0.0  0.0  
kb    T-55AM-1      0.0  0.0  0.0  0.0  0.0  0.0  

[4 rows x 24 columns]

In [53]:
describe_kd(
    subject_target_pairs = [
        ("Sherman Vc", ["85"])
    ]
)

Distance                                           \
                        count      mean       std   min     25%    50%   
Event EnemyVehicle                                                       
k     KV-85               2.0  0.480000  0.452548  0.16  0.3200  0.480   
      SU-85               3.0  0.543333  0.406981  0.10  0.3650  0.630   
      SU-85M              2.0  1.180000  0.028284  1.16  1.1700  1.180   
      T-34-85             6.0  0.883333  0.453373  0.30  0.5950  0.855   
      T-34-85 (D-5T)      4.0  0.322500  0.315317  0.03  0.0600  0.325   
kb    KV-85               4.0  0.552500  0.324487  0.08  0.4775  0.665   
      T-34-85             2.0  0.290000  0.183848  0.16  0.2250  0.290   
      T-34-85 (D-5T)      2.0  0.050000  0.042426  0.02  0.0350  0.050   
      T-34-85 ?           1.0  0.050000       NaN  0.05  0.0500  0.050   

                                   EnemySpeed       ...               \
                         75%   max      count mean  ...    75%   max   
Event EnemyVehicle                                  ...                
k     KV-85           0.6400  0.80        2.0  0.0  ...   0.00   0.0   
      SU-85           0.7650  0.90        3.0  0.0  ...   0.00   0.0   
      SU-85M          1.1900  1.20        2.0  0.0  ...   0.00   0.0   
      T-34-85         1.1450  1.54        6.0  4.0  ...   2.25  21.0   
      T-34-85 (D-5T)  0.5875  0.61        4.0  7.5  ...  11.50  22.0   
kb    KV-85           0.7400  0.80        4.0  0.0  ...   0.00   0.0   
      T-34-85         0.3550  0.42        2.0  0.0  ...   0.00   0.0   
      T-34-85 (D-5T)  0.0650  0.08        2.0  0.0  ...   0.00   0.0   
      T-34-85 ?       0.0500  0.05        1.0  0.0  ...   0.00   0.0   

                     SelfSpeed                                          
                         count mean       std  min  25%  50%  75%  max  
Event EnemyVehicle                                                      
k     KV-85                2.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
      SU-85                3.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
      SU-85M               2.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
      T-34-85              6.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
      T-34-85 (D-5T)       4.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
kb    KV-85                4.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
      T-34-85              2.0  2.0  2.828427  0.0  1.0  2.0  3.0  4.0  
      T-34-85 (D-5T)       2.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  
      T-34-85 ?            1.0  0.0       NaN  0.0  0.0  0.0  0.0  0.0  

[9 rows x 24 columns]

In [54]:
describe_kd(
    subject_target_pairs = [
        ("Firefly", ["85"])
    ]
)

Distance                                             \
                         count      mean       std    min     25%     50%   
Event EnemyVehicle                                                          
k     KV-85               10.0  0.328000  0.242844  0.030  0.1525  0.2800   
      SU-85                2.0  0.440000  0.565685  0.040  0.2400  0.4400   
      SU-85M               3.0  0.740000  0.685055  0.310  0.3450  0.3800   
      T-34-85              9.0  0.440000  0.365923  0.020  0.1100  0.3400   
      T-34-85 (D-5T)      12.0  0.425833  0.255955  0.020  0.2425  0.4575   
      T-34-85 (S53)        3.0  0.720000  0.320780  0.440  0.5450  0.6500   
      T-34-85 Gai          1.0  0.810000       NaN  0.810  0.8100  0.8100   
      T-34-85E             1.0  0.635000       NaN  0.635  0.6350  0.6350   
kb    KV-85                6.0  0.253333  0.243283  0.040  0.1075  0.1650   
      SU-85                1.0  0.800000       NaN  0.800  0.8000  0.8000   
      T-34-85              2.0  0.370000  0.084853  0.310  0.3400  0.3700   
      T-34-85 (D-5T)       5.0  0.172000  0.214523  0.010  0.0300  0.0300   
      T-34-85 Gai          1.0  0.800000       NaN  0.800  0.8000  0.8000   
      T-34-85 No. 215      1.0  0.550000       NaN  0.550  0.5500  0.5500   

                                      EnemySpeed             ...               \
                           75%    max      count       mean  ...    75%   max   
Event EnemyVehicle                                           ...                
k     KV-85            0.47250  0.800       10.0   4.000000  ...   8.25  13.0   
      SU-85            0.64000  0.840        2.0   0.000000  ...   0.00   0.0   
      SU-85M           0.95500  1.530        3.0   4.833333  ...   6.00   8.0   
      T-34-85          0.70000  1.100        9.0   5.222222  ...   4.00  31.0   
      T-34-85 (D-5T)   0.61125  0.820       12.0   8.500000  ...  16.00  27.0   
      T-34-85 (S53)    0.86000  1.070        3.0  13.000000  ...  19.50  29.0   
      T-34-85 Gai      0.81000  0.810        1.0   0.000000  ...   0.00   0.0   
      T-34-85E         0.63500  0.635        1.0   1.500000  ...   1.50   1.5   
kb    KV-85            0.31250  0.700        6.0   1.333333  ...   0.00   8.0   
      SU-85            0.80000  0.800        1.0   0.000000  ...   0.00   0.0   
      T-34-85          0.40000  0.430        2.0   3.000000  ...   4.50   6.0   
      T-34-85 (D-5T)   0.30000  0.490        5.0   5.300000  ...   5.00  20.0   
      T-34-85 Gai      0.80000  0.800        1.0  15.000000  ...  15.00  15.0   
      T-34-85 No. 215  0.55000  0.550        1.0   0.000000  ...   0.00   0.0   

                      SelfSpeed                                                 
                          count      mean       std  min   25%  50%   75%  max  
Event EnemyVehicle                                                              
k     KV-85                10.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0  
      SU-85                 2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0  
      SU-85M                3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0  
      T-34-85               9.0  0.444444  1.333333  0.0  0.00  0.0  0.00  4.0  
      T-34-85 (D-5T)       12.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0  
      T-34-85 (S53)         3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0  
      T-34-85 Gai           1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0  
      T-34-85E              1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0  
kb    KV-85                 6.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0  
      SU-85                 1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0  
      T-34-85               2.0  2.500000  3.535534  0.0  1.25  2.5  3.75  5.0  
      T-34-85 (D-5T)        5.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0  
      T-34-85 Gai           1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0  
      T-34-85 No. 215       1.0  0.000

In [55]:
describe_kd(
    subject_target_pairs = [
        ("Tiger E", ["85"])
    ]
)

Distance                                           \
                         count      mean       std   min     25%    50%   
Event EnemyVehicle                                                        
k     KV-85                5.0  0.574000  0.455664  0.01  0.2900  0.520   
      SU-85                8.0  0.887500  0.731569  0.04  0.5000  0.660   
      SU-85M               6.0  0.490000  0.246901  0.16  0.3250  0.485   
      T-34-85             19.0  0.612105  0.402210  0.02  0.3350  0.620   
      T-34-85 ( D-5T)      1.0  0.300000       NaN  0.30  0.3000  0.300   
      T-34-85 (D-5T)      26.0  0.610000  0.471720  0.04  0.2300  0.520   
      T-34-85 (S-53)       2.0  0.145000  0.162635  0.03  0.0875  0.145   
      T-34-85 (S53)        1.0  0.320000       NaN  0.32  0.3200  0.320   
      T-34-85 Gai          1.0  0.350000       NaN  0.35  0.3500  0.350   
kb    SU-85M               2.0  0.405000  0.261630  0.22  0.3125  0.405   
      T-34-85              6.0  0.273333  0.249773  0.03  0.1200  0.160   
      T-34-85 (D-5T)       2.0  0.310000  0.367696  0.05  0.1800  0.310   
      T-34-85 (S-53)       1.0  1.280000       NaN  1.28  1.2800  1.280   
      T-34-85E             1.0  0.600000       NaN  0.60  0.6000  0.600   

                                    EnemySpeed             ...               \
                          75%   max      count       mean  ...    75%   max   
Event EnemyVehicle                                         ...                
k     KV-85            0.9200  1.13        5.0   0.800000  ...   0.00   4.0   
      SU-85            1.1400  2.12        8.0   1.375000  ...   0.75   8.0   
      SU-85M           0.6900  0.78        6.0   7.666667  ...  14.75  21.0   
      T-34-85          0.7800  1.85       19.0   4.657895  ...   8.00  31.0   
      T-34-85 ( D-5T)  0.3000  0.30        1.0   0.000000  ...   0.00   0.0   
      T-34-85 (D-5T)   0.8925  1.89       26.0   7.153846  ...  11.75  32.0   
      T-34-85 (S-53)   0.2025  0.26        2.0   6.500000  ...   9.75  13.0   
      T-34-85 (S53)    0.3200  0.32        1.0  10.000000  ...  10.00  10.0   
      T-34-85 Gai      0.3500  0.35        1.0   0.000000  ...   0.00   0.0   
kb    SU-85M           0.4975  0.59        2.0   0.000000  ...   0.00   0.0   
      T-34-85          0.4475  0.64        6.0   0.000000  ...   0.00   0.0   
      T-34-85 (D-5T)   0.4400  0.57        2.0  10.000000  ...  15.00  20.0   
      T-34-85 (S-53)   1.2800  1.28        1.0   0.000000  ...   0.00   0.0   
      T-34-85E         0.6000  0.60        1.0   0.000000  ...   0.00   0.0   

                      SelfSpeed                                               \
                          count      mean       std  min    25%   50%    75%   
Event EnemyVehicle                                                             
k     KV-85                 5.0  0.000000  0.000000  0.0  0.000  0.00  0.000   
      SU-85                 8.0  0.000000  0.000000  0.0  0.000  0.00  0.000   
      SU-85M                6.0  0.000000  0.000000  0.0  0.000  0.00  0.000   
      T-34-85              19.0  0.000000  0.000000  0.0  0.000  0.00  0.000   
      T-34-85 ( D-5T)       1.0  0.000000       NaN  0.0  0.000  0.00  0.000   
      T-34-85 (D-5T)       26.0  0.730769  2.676680  0.0  0.000  0.00  0.000   
      T-34-85 (S-53)        2.0  0.250000  0.353553  0.0  0.125  0.25  0.375   
      T-34-85 (S53)         1.0  0.000000       NaN  0.0  0.000  0.00  0.000   
      T-34-85 Gai           1.0  0.000000       NaN  0.0  0.000  0.00  0.000   
kb    SU-85M                2.0  0.000000  0.000000  0.0  0.000  0.00  0.000   
      T-34-85               6.0  2.666667  3.932768  0.0  0.000  1.00  3.500   
      T-34-85 (D-5T)        2.0  0.500000  0.707107  0.0  0.250  0.50  0.750   
      T-34-85 (S-53)        1.0  8.000000       NaN  8.0  8.000  8.00  8.000   
      T-34-85E              1.0  5.000000       NaN  5.0  5.000  5.00  5.000   

                             
           

In [56]:
describe_kd(
    subject_target_pairs = [
        ("Panther D", ["85"])
    ]
)

Distance                                                \
                        count   mean       std   min     25%    50%     75%   
Event EnemyVehicle                                                            
k     T-34-85             4.0  0.365  0.203388  0.20  0.2300  0.305  0.4400   
      T-34-85 (D-5T)      1.0  0.120       NaN  0.12  0.1200  0.120  0.1200   
      T-34-85 (S-53)      1.0  0.480       NaN  0.48  0.4800  0.480  0.4800   
      T-34-85 Gai         2.0  0.670  0.806102  0.10  0.3850  0.670  0.9550   
kb    T-34-85             6.0  0.170  0.131453  0.04  0.0875  0.120  0.2425   
      T-34-85 (D-5T)      2.0  0.155  0.063640  0.11  0.1325  0.155  0.1775   

                           EnemySpeed         ...             SelfSpeed       \
                       max      count   mean  ...    75%  max     count mean   
Event EnemyVehicle                            ...                              
k     T-34-85         0.65        4.0  1.375  ...  2.125  4.0       4.0  0.0   
      T-34-85 (D-5T)  0.12        1.0  0.000  ...  0.000  0.0       1.0  0.0   
      T-34-85 (S-53)  0.48        1.0  0.000  ...  0.000  0.0       1.0  3.0   
      T-34-85 Gai     1.24        2.0  0.750  ...  1.125  1.5       2.0  0.0   
kb    T-34-85         0.38        6.0  0.000  ...  0.000  0.0       6.0  4.0   
      T-34-85 (D-5T)  0.20        2.0  0.000  ...  0.000  0.0       2.0  0.0   

                                                          
                           std  min  25%  50%  75%   max  
Event EnemyVehicle                                        
k     T-34-85         0.000000  0.0  0.0  0.0  0.0   0.0  
      T-34-85 (D-5T)       NaN  0.0  0.0  0.0  0.0   0.0  
      T-34-85 (S-53)       NaN  3.0  3.0  3.0  3.0   3.0  
      T-34-85 Gai     0.000000  0.0  0.0  0.0  0.0   0.0  
kb    T-34-85         7.266361  0.0  0.0  0.0  4.5  18.0  
      T-34-85 (D-5T)  0.000000  0.0  0.0  0.0  0.0   0.0  

[6 rows x 24 columns]

In [57]:
describe_kd(
    subject_target_pairs = [
        ("Panther A", ["85"])
    ]
)

Distance                                           \
                        count      mean       std   min     25%    50%   
Event EnemyVehicle                                                       
k     KV-85               2.0  0.515000  0.021213  0.50  0.5075  0.515   
      T-34-85             7.0  0.358571  0.204322  0.17  0.2250  0.290   
      T-34-85 (D-5T)      6.0  0.941667  0.534132  0.34  0.6475  0.850   
      T-34-85 Gai         1.0  0.670000       NaN  0.67  0.6700  0.670   
kb    T-34-85             1.0  0.100000       NaN  0.10  0.1000  0.100   
      T-34-85 (S-53)      1.0  1.440000       NaN  1.44  1.4400  1.440   

                                   EnemySpeed            ...              \
                         75%   max      count      mean  ...   75%   max   
Event EnemyVehicle                                       ...               
k     KV-85           0.5225  0.53        2.0  6.500000  ...  9.75  13.0   
      T-34-85         0.4650  0.67        7.0  1.714286  ...  3.00   6.0   
      T-34-85 (D-5T)  1.0750  1.88        6.0  5.833333  ...  9.50  12.0   
      T-34-85 Gai     0.6700  0.67        1.0  8.000000  ...  8.00   8.0   
kb    T-34-85         0.1000  0.10        1.0  0.000000  ...  0.00   0.0   
      T-34-85 (S-53)  1.4400  1.44        1.0  0.000000  ...  0.00   0.0   

                     SelfSpeed                                     
                         count mean  std  min  25%  50%  75%  max  
Event EnemyVehicle                                                 
k     KV-85                2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      T-34-85              7.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      T-34-85 (D-5T)       6.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
      T-34-85 Gai          1.0  0.0  NaN  0.0  0.0  0.0  0.0  0.0  
kb    T-34-85              1.0  0.0  NaN  0.0  0.0  0.0  0.0  0.0  
      T-34-85 (S-53)       1.0  0.0  NaN  0.0  0.0  0.0  0.0  0.0  

[6 rows x 24 columns]

## Remarks on Exploratory Analysis

- More data is needed. The average kill distance for the T-44-100 is .76 kilometers while the average killed-by distance is 1.08. Further, the kill and killed-by distances of the T-44-100 are greater than for the T-54 series (1949 and 1947) despite the 1947 having better armor and the 1949 having better shells.

- There is a presumed reason for the T-54 (1949) being killed at a longer range (0.75) than the T-54 (1947) despite the 1949 series having better ammunition for long range. Since the 1949 series is generally better than the 1947, it is spawned first. Once killed, the enemy players have an approximate idea of where the player is and manage to get close more often thanks to knowledge gained while in the 1949. This is an unproven explanation and may require an additional column/parameter to be verified.


Update
- As of August 17, 2021, the engagment distance between the Centurion and T-54 series has changed. It has become more like the expected values: kills at .72 km, deaths at .435 (1947) and .491 (1949).

Update

- As of October 13, 2021, ratio vs. T-54 (1949) reached ~3.146. 129/41.
- Ratio versus T-54 (1947) mostly unchanged: ~2.214. 93/42.

Update: KD Review 2021-10-27

#### Centurion Mk. 10
- All replays earlier than Oct. 26 reviewed.
- Ratio versus T-54 (1947) mostly unchanged despite mostly losses since Oct. 13 gains.
- Ratio versus T-54 (1949) has insignificant improvement: 3.200 versus previous 3.146
- Ranges versus T-54 (1947) unchanged: kills at .720 (no change), deaths at .434 (-.001).
- Ranges versus T-54 (1949) slightly changed: kills at .736 (+.016), deaths at .487 (+.052).

Update: KD Review 2021-11-03

#### Cheiftain Mk. 5
- Ratio for Chieftain Mk. 5 meets the 1-1 goal versus T-64A (1971): 17/13 = ~1.308.
- It is one kill short of 2-1 goal vs. T-62M-1: (11/6 = 1.8333...).
- Exceeds 2-1 goal vs. T-55AM-1: 10/2, 5.0. Manual review showed 12/3 = 4.

In [58]:
print("Done")

Done
